In [1]:
import scipy as sp
import pandas as pd
import os
import numpy as np
import h5py

In [2]:
folder_path = "Experimental_data_TD"
file_name_list = os.listdir(folder_path)
filename_path = [os.path.join(folder_path, file_name) for file_name in file_name_list if file_name.endswith(".xlsx")]
print(filename_path)

['Experimental_data_TD/TDset_oxygen_Tw-20.xlsx', 'Experimental_data_TD/TDset_oxygen_Tw25.xlsx', 'Experimental_data_TD/TDset_oxygen_Tw5.xlsx', 'Experimental_data_TD/TDset_oxygen_Tw50.xlsx']


In [3]:
df = pd.read_excel(filename_path[0])
print(df.head())

   Pressure(torr)  Current(mA)  Flow(sccm)  Twall  fracCO2  fracO2  \
0             0.4           20       7.791    -20        0       1   
1             0.6           20       7.791    -20        0       1   
2             0.8           20       7.791    -20        0       1   
3             1.5           20       7.791    -20        0       1   
4             2.0           20       7.791    -20        0       1   

   lossFreqExp  recProbExp     TgasExp  ENest(Td)      TnwExp  \
0    16.604905    0.000560  268.941983  71.140809  257.571755   
1    20.238391    0.000674  276.610425  65.604231  259.718919   
2    22.052369    0.000726  284.986092  61.549791  262.064106   
3    26.900785    0.000850  302.968079  53.963316  267.099062   
4    27.164979    0.000832  319.053573  51.823836  271.603000   

           OmeanExp  Unnamed: 12  Unnamed: 13   TgasExp50  ENest50(Td)  \
0  1493906152913230          NaN          NaN  338.941983    71.140809   
1  2482489172052840          NaN        

In [4]:

print(list(df.columns))

schema = {
    "Pressure(torr)": "pressure",
    "Current(mA)": "current",
    "Twall": "Tw",
    "TnwExp": "Tnw", 
    "OmeanExp": "O",
    "COmeanExp": "CO",
    "recProbExp": "gamma_exp"
}

data_tensor = {name: [] for name in schema.values()}


# print(data_tensor)


pressure_vec = df["Pressure(torr)"].to_numpy()
print(pressure_vec)


for file_name in filename_path:
    df = pd.read_excel(file_name)
    
    for name_exp, name_sim in schema.items():
        
        if name_exp in df.columns:
            vec_aux = df[name_exp].to_numpy()
            data_tensor[name_sim].append(vec_aux[~np.isnan(vec_aux)])


for name in schema.values():
    if len(data_tensor[name]) > 0: 
        data_tensor[name] = np.concatenate(data_tensor[name])

print(data_tensor['pressure'])
print(data_tensor['Tw'])

print(data_tensor.keys())

### write data with hdf5 file 

['Pressure(torr)', 'Current(mA)', 'Flow(sccm)', 'Twall', 'fracCO2', 'fracO2', 'lossFreqExp', 'recProbExp', 'TgasExp', 'ENest(Td)', 'TnwExp', 'OmeanExp', 'Unnamed: 12', 'Unnamed: 13', 'TgasExp50', 'ENest50(Td)', 'Unnamed: 16', 'Tgasbefore']
[0.4 0.6 0.8 1.5 2.  3.  0.4 0.6 0.8 1.  1.5 2.  3.  5.  7.5]
[0.4 0.6 0.8 1.5 2.  3.  0.4 0.6 0.8 1.  1.5 2.  3.  5.  7.5 0.4 0.6 0.8
 1.  1.5 2.  3.  5.  0.4 0.6 0.8 1.  1.5 2.  3.  5.  7.5 0.4 0.6 0.8 1.
 1.5 2.  3.  5.  0.4 0.6 0.8 1.  1.5 2.  3.  5.  7.5 0.4 0.6 0.8 1.  1.5
 2.  3.  5.  0.4 0.6 0.8 1.  1.5 2.  3.  5.  7.5]
[-20. -20. -20. -20. -20. -20. -20. -20. -20. -20. -20. -20. -20. -20.
 -20.  25.  25.  25.  25.  25.  25.  25.  25.  25.  25.  25.  25.  25.
  25.  25.  25.  25.   5.   5.   5.   5.   5.   5.   5.   5.   5.   5.
   5.   5.   5.   5.   5.   5.   5.  50.  50.  50.  50.  50.  50.  50.
  50.  50.  50.  50.  50.  50.  50.  50.  50.  50.]
dict_keys(['pressure', 'current', 'Tw', 'Tnw', 'O', 'CO', 'gamma_exp'])


In [5]:

###! write data in hdf5 file 

hdf5_file = "test_exp_data.hdf5"

with h5py.File(hdf5_file, 'w') as f:
    
    for name, vec in data_tensor.items():
        f.create_dataset(name, data=vec)
    
    f.close()


In [10]:


### read data to check 

with h5py.File(hdf5_file, 'r') as f:
    print(f.keys())
    
    print(f['CO'][:])
    print(f['O'][:])
    print(f['Tnw'][:])
    print(f['current'][:])
    print(f['gamma_exp'][:])
    print(f['pressure'][:])


<KeysViewHDF5 ['CO', 'O', 'Tnw', 'Tw', 'current', 'gamma_exp', 'pressure']>
[]
[1.49390615e+15 2.48248917e+15 3.04436238e+15 4.81483190e+15
 5.66034564e+15 5.87322990e+15 1.58747774e+15 2.85627673e+15
 3.45218295e+15 4.51424757e+15 5.71270024e+15 6.57868821e+15
 7.41470218e+15 7.70721080e+15 6.97527853e+15 1.43279928e+15
 2.62540281e+15 3.64701206e+15 4.57951835e+15 6.13949515e+15
 7.12628229e+15 7.65636648e+15 6.80244088e+15 1.70900118e+15
 3.08485244e+15 4.34623353e+15 5.24423561e+15 6.87802883e+15
 8.02849464e+15 9.26973051e+15 9.82828986e+15 7.05322789e+15
 1.80687919e+15 3.06223611e+15 4.10721898e+15 4.99459996e+15
 6.59124649e+15 7.73113278e+15 8.91475344e+15 8.27504332e+15
 1.97761540e+15 3.50442408e+15 4.82611243e+15 6.01068056e+15
 7.77474960e+15 9.13103058e+15 1.04939622e+16 1.09607510e+16
 1.00147976e+16 1.22064007e+15 2.32028959e+15 3.40846622e+15
 4.30696647e+15 6.10308420e+15 7.27253759e+15 8.38861398e+15
 7.84230376e+15 1.34933921e+15 2.60814796e+15 3.84926420e+15
 5.106

In [ ]:

###  create a folder with outpu

os.makedirs()
